In [1]:
import numpy as np 

In [2]:
def OMP(y, D, K):

    S = []
    x = np.zeros(D.shape[1])

    R = np.copy(y)

    for _ in range(K):
        corr = D.T @ R

        S.append(np.argmax(np.abs(corr)))
        dic = D[:,S]
        x_hat = np.linalg.inv(dic.T @ dic) @ dic.T @ y
        x[S] = x_hat

        R = y - D @ x

    return x

In [3]:
def vecOMP(Y, D, K):
    batch_size, _ = Y.shape
    dictionary_dim = D.shape[1]
    
    S = np.zeros((K, batch_size), dtype=int)
    X = np.zeros((batch_size, dictionary_dim))
    R = np.copy(Y)
    
    for k in range(K):

        corr = np.abs(R @ D)

        # Masking the previously selected atoms
        mask_1 = np.zeros_like(corr, dtype=bool)
        mask_1[np.arange(batch_size)[:, None], S[:k,:].transpose(1,0)] = True
        corr[mask_1] = -np.inf

        # Selected highest correlated atoms
        selected = np.argmax(corr, axis=1)
        S[k, :] = selected

        # Solve least square problem
        dics = np.linalg.pinv(D[:, S[:k+1,:]].transpose(2,0,1))
        X_hat = np.matmul(dics, Y  [:,:,None])[:,:,0]

        # Update the estimate
        mask_2 = np.zeros_like(X, dtype=bool)
        mask_2[np.arange(batch_size)[:, None], S[:k+1,:].transpose(1,0)] = True
        X[mask_2] = X_hat.reshape(-1)

        # Update the residuals
        R = Y - (X @ D.T)
    
    return X, S


In [4]:
Y = np.random.rand(100, 20)  
D = np.eye(20)
K = 5  

X, S = vecOMP(Y, D, K)

In [7]:
X.shape

(100, 20)

In [5]:
X.shape

(100, 20)